In [4]:
import pandas as pd
import math
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

from scipy.interpolate import interp1d #pre written interpolation function
from scipy.interpolate import griddata

In [5]:
BETA = .9
GAMMA = .5772 #euler's constant
THETA = 1.1
N = 7
BIDS = np.arange(1,16)

In [6]:
def enum_other_states(n, min_s, max_s):
    """enumerate the state. for all palyers except player i
    (1,1,2) is equivalent to (1,2,1) from firm 1's pespective"""
    
    states_i = range(min_s,max_s)
    if n <=1:
        states = [[i] for i in states_i]
        return states
    else:
        states = []
        states_j = enum_other_states(n-1,min_s,max_s)
        for i in states_i:
            for j in states_j:
                if j[0] >= i:
                    state_ij = [i] + j 
                    states.append(state_ij)
        return states

In [68]:
class States:
    """class for dealing with the states
    these things are apparently really complex
    there are just a ton of them"""
    
    my_states = None #type nparray
    other_states = None #type np array 2d
    n = 0 #number of players
    max_state = 0
    min_state = 0
    

    def __init__(self, n, min_s,max_s):
        """initialize important class attributes"""
        self.n, self.min_states, self.max_states = n, min_s, max_s
      
    
    def gen_states(self):
        """compute all relevant states for firm i
        the trick is ommiting equivalent states
        this is taken care of in the helper function"""
        
        print 'yo'
        
        #avoid typing self a million times
        n, min_s, max_s  = self.n, self.min_states, self.max_states
        
        #setup other states
        other_states =  np.array(enum_other_states(n-1,min_s,max_s))
        num_others = other_states.shape[0]
        
        #initialize my states based on this
        my_states = np.arange(min_s,max_s)
        my_states = np.repeat(my_states,num_others )
        self.my_states = my_states
        
        #copy other states enough times
        other_states = other_states.reshape((1,num_others,n-1))
        other_states = np.tile(other_states,(max_s - min_s,1,1))  
        other_states = other_states.reshape(num_others*(max_s - min_s),n-1)
        self.other_states = other_states
    
    
    
    def make_states(self, my_states, other_states):
        """special initializer for states
        ensures they are not too big or too small"""
        
        assert np.array(other_states).shape[1] == (self.n -1)
        
        n, min_s, max_s  = self.n, self.min_states, self.max_states
        
        #make a copy to avoid side effects
        new_states = States(n,min_s,max_s)
        
        #ensure other my states are good
        my_states = np.array(my_states)
        my_states = np.minimum(max_s - 1, np.array(my_states))
        new_states.my_states = np.maximum(min_s, np.array(my_states))
        
        #ensure other states are good
        other_states = np.sort(other_states,axis=1)
        other_states = np.minimum(max_s - 1, np.array(other_states))
        new_states.other_states = np.maximum(min_s, np.array(other_states))
        
        return new_states
    
    
    def get_len(self):
        return len(self.other_states)
    
    
    def get_all(self):
        """return all the states"""
        reshaped_states = self.my_states.reshape((len(self.my_states),1))
        return np.concatenate( (reshaped_states, self.other_states), axis=1 )

In [67]:
def create_function(states, values):
    """returns a function
    the function uses the grid data to figure out return value"""
    f = lambda eval_states: griddata(states.get_all(), values, 
                                     eval_states.get_all(), method='linear')
    return f


states = States(3,1,3)
states.gen_states()
new_states = states.make_states([1,2],[[1.5,1],[1,5]])
values = np.concatenate([3*np.ones(states.get_len()/2),np.ones(states.get_len()/2)])
print len(values)
print states.get_len()
policy = create_function(states, values)
print policy(new_states)

yo
6
6
[3. 1.]


In [22]:
def calc_bids(policy, states, n):
    """return the other players bids
    given the state"""
    
    other_bids = []
    for i in range(1,n):
        other_states = states.get_all()
        my_other = other_states[:,i]
        other = np.delete(other_states, i, axis=1)
        other_states = States(my_other, other)
        other_bids.append( policy(other_states) )
    return np.array(other_bids).transpose()

[[1 1 1]
 [1 1 2]
 [1 2 2]
 [2 1 1]
 [2 1 2]
 [2 2 2]]
[[5 5]
 [5 5]
 [5 5]
 [5 5]
 [5 5]
 [5 5]]


In [ ]:


    

    
    
def cost(states, shock):
    """ states are all states, the shock is just an integer"""
    return THETA*states + shock


def profit(contract, my_bid, other_bids, state, shock):
    """this returns the same dimension as my bid
    
    will need to tile bids if evaluating multiple at once"""
    return contract*( my_bid < np.min(other_bids) )*( my_bid - cost(state,shock) )  

In [ ]:
def backlog(contract, my_bid,  other_bids, state):
    """this returns the same dimension as my bid
    
     will need to tile bids if evaluating multiple at once"""
    
    ##tile the other bids so that I can evalute this expression for each bid
    #tile the states so I can evaluate this expression for each bid at each state
    
    return np.minimum( np.max(STATES), contract*(  my_bid < np.min(other_bids) ) + state )

In [ ]:
def operator(initial_value, initial_policy):
    """Calculate the value of the 
    Bellman operator in the value function
    
    update the value function and the policy function
    for player i functions
    
    since we are looking for symmetric eq, there is really just 1
    policy and value to keep track of"""
    
    #initialize contract and state to 0
    contract = 1
    shock = 0
    
    #possible bids for all states, again 1 player
    bids = BIDS.reshape((len(BIDS), 1, ))
    bids = np.tile(bids,(1,len(STATES)))
    
    
    #calculate value of each bid and take max in each state
    v_next = profit(contract,bids,other_bids,states,shock)
    v_next = v_next + BETA*value(backlog(contract, bids, other_bids, states))
    
    #find maximum and argmax
    policy = np.argmax(v_next,axis=0)
    policy = BIDS[policy]
    v_next = np.max(v_next,axis=0)
    
    return policy, v_next



#initialize policy as something else
foo_policy = lambda state: 5
foo_value = lambda state: 0

operator(foo_value, foo_policy)

In [ ]:
def  value_function(initial_value, error, maxiter):
    """calculate the value function in a symmetric
        markov perfect equilibria"""
    
    #initialize loop variables
    v0 = np.ones(len(states))
    v_next = operator(a, theta1, cost, v0, v1)
    
    while  maxiter >= 0 and ( 
            np.abs(v0 - v0_next ).max() > error
           or np.abs(v1 - v1_next ).max()  > error ):
        
        #iterate loop variables for each player
        for i in range(N):
            v0, v1 = v0_next, v1_next
            v0_next, v1_next = operator(a, theta1, cost, v0, v1)
            maxiter = maxiter -1

    return  value